In [1]:
from aicsimageio import AICSImage, imread
from aicsimageio.writers import PngWriter
from pathlib import Path
import napari
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import torch
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from tqdm import tqdm

In [2]:
def get_paths(dir_path, ext):
    return [str(path) for path in Path(dir_path).rglob('*.{}'.format(ext))]

In [3]:
ext_tiff = 'czi'
folder_tiff = '../../../datasets/microscopy_luke'

In [4]:
files = get_paths(folder_tiff, ext_tiff)

In [5]:
len(files)

1541

In [6]:
test = AICSImage(files[0])
test = test.get_image_data("CZYX", S=0, T=0).squeeze(1)
test.shape

/home/victor/anaconda3/envs/sr/lib/python3.7/site-packages/aicsimageio/transforms.py:178: UserWarning: Data has dimension B with depth 1, assuming B=0 is the desired value, if not the case specify B=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "
/home/victor/anaconda3/envs/sr/lib/python3.7/site-packages/aicsimageio/transforms.py:178: UserWarning: Data has dimension V with depth 1, assuming V=0 is the desired value, if not the case specify V=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "


(5, 1024, 1024)

In [11]:
def AICS_to_grayscale(img_path, chanel, stack=False):
    img = AICSImage(img_path)
    img = img.get_image_data("CZYX", S=0, T=0)
    img = img.squeeze(1)

    if img.shape[0] != 5:
        print('INVALID SHAPE', img_path, img.shape[0])
    
    if img.shape[0] >= chanel:
        img = np.asarray(img)[chanel]
    else:
        print('INVALID SHAPE', img_path, img.shape[0])
        return None
    
    if stack:
        img = np.stack([img, img, img], 2)

    return img

In [12]:
for f in tqdm(files):
    _ = AICS_to_grayscale(f, 2)

 65%|██████▌   | 1004/1541 [03:17<01:30,  5.94it/s]

INVALID SHAPE ../../../datasets/microscopy_luke/20180506/day5 PWPI-GFP+PWPI-mCherry/day5 PWPI-GFPPWPI-mCherry-23.czi 1
INVALID SHAPE ../../../datasets/microscopy_luke/20180506/day5 PWPI-GFP+PWPI-mCherry/day5 PWPI-GFPPWPI-mCherry-23.czi 1


100%|██████████| 1541/1541 [05:06<00:00,  5.03it/s]


In [ ]:
for i in range(10):
    img = AICS_to_grayscale(files[i], 3, stack=True)
    plt.figure()
    plt.imshow(img)

In [13]:
def downsample_save(files, chanel, scale_factor, ds_dir):
    
    lr_dir = os.path.join(ds_dir, 'lr_{}x'.format(scale_factor))
    hr_dir = os.path.join(ds_dir, 'hr'.format(scale_factor))
    
    os.makedirs(lr_dir, exist_ok=True)
    os.makedirs(hr_dir, exist_ok=True)
    
    trans = transforms.ToTensor()
    
    for i, f in enumerate(tqdm(files, 'saving')):
        hr = AICS_to_grayscale(files[i], chanel, stack=True)
        
        hr_name = os.path.join(hr_dir, 'sample_{}.png'.format(i))        
        lr_name = os.path.join(lr_dir, 'sample_{}.png'.format(i))
        
        if hr is not None and not (os.path.exists(hr_name) and os.path.exists(lr_name)):
            
            hr = trans(hr).unsqueeze(0)

            lr = F.interpolate(hr, scale_factor=scale_factor, mode='bicubic', align_corners=True)

            
            print(hr.shape)
            hr_pil = to_pil_image(hr.squeeze(0))
            lr_pil = to_pil_image(lr.squeeze(0))

#             if not os.path.exists(hr_name):
#                 hr_pil.save(hr_name)

#             lr_pil.save(lr_name)

In [14]:
downsample_save(files, chanel=2, scale_factor=1/4, ds_dir='data/czi')

saving:   0%|          | 0/1541 [00:00<?, ?it/s]/home/victor/anaconda3/envs/sr/lib/python3.7/site-packages/aicsimageio/transforms.py:178: UserWarning: Data has dimension B with depth 1, assuming B=0 is the desired value, if not the case specify B=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "
/home/victor/anaconda3/envs/sr/lib/python3.7/site-packages/aicsimageio/transforms.py:178: UserWarning: Data has dimension V with depth 1, assuming V=0 is the desired value, if not the case specify V=x where x is an integer, list, tuple, range, or slice.
  f"Data has dimension {dim} with depth {data.shape[dim_index]}, "
/home/victor/anaconda3/envs/sr/lib/python3.7/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. 

torch.Size([1, 3, 1024, 1024])


saving:   0%|          | 2/1541 [00:00<06:13,  4.12it/s]

torch.Size([1, 3, 1024, 1024])


saving:   0%|          | 3/1541 [00:00<06:12,  4.13it/s]

...

torch.Size([1, 3, 1024, 1024])


saving:   5%|▌         | 84/1541 [00:19<05:33,  4.37it/s]

torch.Size([1, 3, 1024, 1024])


saving:   5%|▌         | 84/1541 [00:19<05:42,  4.26it/s]


KeyboardInterrupt: 

In [ ]:
downsample_save(files, chanel=2, scale_factor=1/8, ds_dir='data/czi')

In [ ]:
downsample_save(files, chanel=2, scale_factor=1/16, ds_dir='data/czi')